In [1]:
import pandas as pd
import datetime as dt
import warnings
import os
warnings.filterwarnings("ignore")

In [8]:
pip install taichi


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/83.2 MB ? eta -:--:--
   - -------------------------------------- 2.1/83.2 MB 29.1 MB/s eta 0:00:03
   -- ------------------------------------- 4.2/83.2 MB 12.0 MB/s eta 0:00:07
   --- ------------------------------------ 7.3/83.2 MB 11.9 MB/s eta 0:00:07
   --- ------------------------------------ 7.3/83.2 MB 11.9 MB/s eta 0:00:07
   ---- ----------------------------------- 8.4/83.2 MB 9.0 MB/s eta 0:00:09
   ---- ----------------------------------- 9.4/83.2 MB 8.3 MB/s eta 0:00:09
   ------ --------------------------------- 12.6/83.2 MB 8.7 MB/s eta 0:00:09
   ------ --------------------------------- 13.6/83.2 MB 9.2 MB/s eta 0:00:08
   ------- -------------------------------- 14.7/83.2 MB 8.6 MB/s eta 0:00:08
   --------- ------------------------------ 19.7/83.2 MB 9.5 MB/s eta 0:00:07
   ----------

In [1]:
import taichi as ti

ti.init()

x = ti.field(ti.f32, shape=(4, 4))

@ti.kernel
def fill():
    for i, j in x:
        x[i, j] = i * 0.1 + j

fill()
print(x)


[Taichi] version 1.7.2, llvm 15.0.1, commit 0131dce9, win, python 3.12.4
[Taichi] Starting on arch=x64
[[0.  1.  2.  3. ]
 [0.1 1.1 2.1 3.1]
 [0.2 1.2 2.2 3.2]
 [0.3 1.3 2.3 3.3]]


In [2]:
#################################### Load the stock data ####################################
data = pd.read_csv('LT_spot_data.csv')
data = data.drop(columns=['Unnamed: 0'])
data['Timestamp'] = data['open_timestamp']
data = data[['Timestamp', 'open', 'high', 'low', 'close']]
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['date'] = data['Timestamp'].dt.date
data['time'] = data['Timestamp'].dt.time
# remove the second part of the time
data['time'] = data['time'].apply(lambda x: x.replace(second=0))
################# convert the time to 15 minutes interval #################
data['time'] = data['time'].apply(lambda x: x.replace(minute=(x.minute//15)*15))
data 

,Timestamp,open,high,low,close,date,time
0,2018-10-10 09:15:01,1227.0,1231.00,1223.70,1228.95,2018-10-10,09:15:00
1,2018-10-10 09:30:00,1228.9,1236.50,1228.00,1236.00,2018-10-10,09:30:00
2,2018-10-10 09:45:00,1236.0,1239.25,1234.50,1238.00,2018-10-10,09:45:00
3,2018-10-10 10:00:01,1237.7,1239.55,1233.10,1235.35,2018-10-10,10:00:00
4,2018-10-10 10:15:10,1235.0,1236.00,1232.55,1234.80,2018-10-10,10:15:00
...,...,...,...,...,...,...,...
33174,2024-02-29 14:15:02,3471.1,3471.70,3460.15,3461.00,2024-02-29,14:15:00
33175,2024-02-29 14:30:00,3460.0,3468.90,3451.90,3468.40,2024-02-29,14:30:00
33176,2024-02-29 14:45:01,3469.3,3471.80,3458.65,3462.80,2024-02-29,14:45:00
33177,2024-02-29 15:00:00,3464.0,3483.10,3456.05,3483.10,2024-02-29,15:00:00


In [3]:
data['time'].unique()

array([datetime.time(9, 15), datetime.time(9, 30), datetime.time(9, 45),
       datetime.time(10, 0), datetime.time(10, 15), datetime.time(10, 30),
       datetime.time(10, 45), datetime.time(11, 0), datetime.time(11, 15),
       datetime.time(11, 30), datetime.time(11, 45), datetime.time(12, 0),
       datetime.time(12, 15), datetime.time(12, 30),
       datetime.time(12, 45), datetime.time(13, 0), datetime.time(13, 15),
       datetime.time(13, 30), datetime.time(13, 45), datetime.time(14, 0),
       datetime.time(14, 15), datetime.time(14, 30),
       datetime.time(14, 45), datetime.time(15, 0), datetime.time(15, 15)],
      dtype=object)

In [4]:
####################### store the data in a csv file #######################
base_dir = r"C:\Users\arushi\OneDrive\Desktop\data_fol\15min_data\lt_data"
data.to_csv(os.path.join(base_dir, 'lt_spot_data.csv'))

In [5]:
############################################## Load the future data ##############################################
data = pd.read_csv('LT_fut_data.csv')
data = data.drop(columns=['Unnamed: 0'])
data['Timestamp'] = data['open_timestamp']
data = data[['Timestamp', 'open', 'high', 'low', 'close','Ticker', 'Expiry']]
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['date'] = data['Timestamp'].dt.date
data['time'] = data['Timestamp'].dt.time
# remove the second part of the time
data['time'] = data['time'].apply(lambda x: x.replace(second=0))
################# convert the time to 15 minutes interval #################
data['time'] = data['time'].apply(lambda x: x.replace(minute=(x.minute//15)*15))
data 

,Timestamp,open,high,low,close,Ticker,Expiry,date,time
0,2018-10-10 09:15:01,1217.00,1221.50,1214.10,1219.20,LT18OCTFUT,2018-10-25,2018-10-10,09:15:00
1,2018-10-10 09:30:06,1219.25,1226.70,1218.85,1226.25,LT18OCTFUT,2018-10-25,2018-10-10,09:30:00
2,2018-10-10 09:45:06,1226.70,1229.90,1224.65,1227.65,LT18OCTFUT,2018-10-25,2018-10-10,09:45:00
3,2018-10-10 10:00:04,1227.60,1229.80,1223.25,1225.10,LT18OCTFUT,2018-10-25,2018-10-10,10:00:00
4,2018-10-10 10:15:10,1225.40,1226.00,1223.05,1224.60,LT18OCTFUT,2018-10-25,2018-10-10,10:15:00
...,...,...,...,...,...,...,...,...,...
33174,2024-02-29 14:15:00,3475.00,3475.00,3465.00,3465.05,LT24FEBFUT,2024-02-29,2024-02-29,14:15:00
33175,2024-02-29 14:30:01,3465.05,3467.55,3454.45,3467.55,LT24FEBFUT,2024-02-29,2024-02-29,14:30:00
33176,2024-02-29 14:45:00,3468.05,3469.90,3460.45,3464.40,LT24FEBFUT,2024-02-29,2024-02-29,14:45:00
33177,2024-02-29 15:00:00,3464.45,3478.75,3458.45,3477.75,LT24FEBFUT,2024-02-29,2024-02-29,15:00:00


In [6]:
data['time'].unique()

array([datetime.time(9, 15), datetime.time(9, 30), datetime.time(9, 45),
       datetime.time(10, 0), datetime.time(10, 15), datetime.time(10, 30),
       datetime.time(10, 45), datetime.time(11, 0), datetime.time(11, 15),
       datetime.time(11, 30), datetime.time(11, 45), datetime.time(12, 0),
       datetime.time(12, 15), datetime.time(12, 30),
       datetime.time(12, 45), datetime.time(13, 0), datetime.time(13, 15),
       datetime.time(13, 30), datetime.time(13, 45), datetime.time(14, 0),
       datetime.time(14, 15), datetime.time(14, 30),
       datetime.time(14, 45), datetime.time(15, 0), datetime.time(15, 15)],
      dtype=object)

In [7]:
####################### store the data in a csv file #######################
base_dir = r"C:\Users\arushi\OneDrive\Desktop\data_fol\15min_data\lt_data"
data.to_csv(os.path.join(base_dir, 'lt_future_data.csv'))